In [1]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
import pickle
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]
#_CSV_DATA_FILENAME: str = os.environ["_csv_data_filename"]



import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [2]:
# import all your modules here
import time
import json
import pandas as pd

import config_v2 as cfg
from library_ubidots_v2 import Ubidots as ubi

Baseline en config_v2.py: ['2025-05-01', '2025-08-25']
Study en config_v2.py: ['2025-08-25', '2025-09-01']


In [3]:
#blob_name = r"data/data_weekly_report.pkl"
#downloader.upload_pkl_file(bancolombia_pkl,blob_name, blob_container)

In [4]:
# set your constants here
baseline=cfg.BASELINE
study=cfg.STUDY
#study = ['2025-01-01', '2025-02-01']

In [5]:
print(study)
print(baseline)

['2025-08-25', '2025-09-01']
['2025-05-01', '2025-08-25']


In [6]:
df_devices = ubi.get_available_devices_v2(label='bancolombia-f2', level='group', page_size=1000)
df_vars = ubi.get_available_variables(list(df_devices['device_id']))

In [7]:
df_devices

,device_name,device_label,device_id
0,BC 83 - Miramar,bc-83---miramar,67d1c0f06fc439000ea25671
1,BC 795 - Cañaveral,bc-795---canaveral,67b73809266c0f2abaa4dd54
2,BC 322 - San Gil,bc-322---san-gil,67b737d619aa0e22e0e5f84e
3,BC 325 - Santuario,bc-325---santuario,67b7376ddbb3442aff4eab47
4,BC 43 - Puente Aranda,bc-43---puente-aranda,67b73749d50da42a53390cf2
5,BC 602 - UGI,bc-602---ugi,67b7371fdbb3442c70975d96
6,BC 68 - Ibague,bc-68---ibague,67b736f57768142929f1f1fa
7,BC 57 - Villavicencio Principal,bc-57---villavicencio-principal,67b736cf7768142929f1f1f9
8,BC - Metropolitan,bc---metropolitan,668eac0c4e426c000d962c8d
9,BC - Jardin Plaza,bc---jardin-plaza,6644d727097b8c000d7da004


In [8]:
#df_vars = ubi.get_available_variables(['65b00756bd7602000b6876b8'])

In [9]:
df_vars

,variable_id,variable_label,device_id
0,67f43e3237c582000eece7a0,consumo-domingo,67d1c0f06fc439000ea25671
1,67f43e1f5e5d4f000e87d997,consumo-sabado,67d1c0f06fc439000ea25671
2,67f43e0841d651000e8c3e98,consumo-semana,67d1c0f06fc439000ea25671
3,67f43df76ef647000f2f4bc6,potencia-diaria-promedio,67d1c0f06fc439000ea25671
4,67f43ddf6ef647001131946c,tipo-dia,67d1c0f06fc439000ea25671
...,...,...,...
1584,6488c1585f39c9005afa4051,potencia-inductiva,6488bcaf874ff7000ec039ef
1585,6488c1585f39c9005afa4050,front-potencia-activa,6488bcaf874ff7000ec039ef
1586,6488c1585f39c9005afa404f,energia-aparente-consumida,6488bcaf874ff7000ec039ef
1587,6488c1585f39c9005afa404e,energia-reactiva-capacitiva-consumida,6488bcaf874ff7000ec039ef


In [10]:
df_vars = df_vars[df_vars['variable_label'].isin(cfg.WHITELISTED_VAR_LABELS)]
VAR_IDS_TO_REQUEST = list(df_vars['variable_id'])
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [11]:
#leer el archivo data_weekly_report.pkl y guardarlo en un dataframe llamado df1
df1 = pd.read_pickle(r'C:\Digitalizacion\Bancolombia\CB_informe_Fase_2\data\data_weekly_report.pkl')


In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42751971 entries, 2024-10-07 00:00:00-05:00 to 2025-08-19 00:03:00-05:00
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   value        float64
 1   variable     object 
 2   device       object 
 3   device_name  object 
dtypes: float64(1), object(3)
memory usage: 1.6+ GB


In [13]:
#study = ['2025-06-02', '2025-06-09']

In [14]:
CHUNK_SIZE = 10
DATE_INTERVAL_REQUEST = {'start': study[0], 'end': study[1]}
df = None
lst_responses = []
n_vars = len(VAR_IDS_TO_REQUEST)
print(f"Making request for the following interval: Study:{study[0]}, Study:{study[1]}")
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    chunk = VAR_IDS_TO_REQUEST[idx_start:idx_end]
    response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST, 
    )
    if response.status_code == 204 or response.status_code >= 500:
        print(f"Empty response for chunk {idx}")
        time.sleep(10)
        response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST,)
    current_idx = idx_end+1
    if (current_idx > n_vars):
        current_idx = n_vars
    print(f"Progress: {100*(current_idx)/n_vars:0.1f}%")
    print(f"Response status code: {response.status_code}")
    if (response.status_code != 204) and  (len(response.json()['results']) >0 ):
        lst_responses.append(response)
    else: 
        print(f"Empty response for chunk {idx}")
df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)

Making request for the following interval: Study:2025-08-25, Study:2025-09-01
Progress: 2.5%
Response status code: 200
Progress: 4.8%
Response status code: 200
Progress: 7.0%
Response status code: 200
Progress: 9.3%
Response status code: 200
Progress: 11.6%
Response status code: 200
Progress: 13.8%
Response status code: 200
Progress: 16.1%
Response status code: 200
Progress: 18.4%
Response status code: 200
Progress: 20.6%
Response status code: 200
Progress: 22.9%
Response status code: 200
Progress: 25.2%
Response status code: 200
Progress: 27.4%
Response status code: 200
Progress: 29.7%
Response status code: 200
Progress: 32.0%
Response status code: 200
Progress: 34.2%
Response status code: 200
Progress: 36.5%
Response status code: 200
Progress: 38.8%
Response status code: 200
Progress: 41.0%
Response status code: 200
Progress: 43.3%
Response status code: 200
Progress: 45.6%
Response status code: 200
Progress: 47.8%
Response status code: 200
Progress: 50.1%
Response status code: 200
Pr

In [15]:
#Unir el dataframe dfprogress: 30.5%Response status code: 200Progress: 34.8%Response status code: 200Progress: 39.1%Response status code: 200Progress: 43.3%Response status code: 200 y df1 en uno que se llame df_full / esto es para mas adelante
df = pd.concat([df1, df], axis=0, ignore_index=False)

# para quitar el indixe de "datatime" y volverlo columna 
df.reset_index(inplace=True)

# Quitar duplicados 
df = df.drop_duplicates()

# Establecer la columna "datetime" como el índice
df.set_index('datetime', inplace=True)


In [16]:
pd.to_pickle(df, project_path / 'data'/ _PICKLED_DATA_FILENAME)

In [17]:
df.index[(df.index.month == 1)&(df.index.year == 2025)].unique()

DatetimeIndex(['2025-01-07 00:00:00-05:00', '2025-01-06 23:57:00-05:00',
               '2025-01-06 23:54:00-05:00', '2025-01-06 23:51:00-05:00',
               '2025-01-06 23:48:00-05:00', '2025-01-06 23:45:00-05:00',
               '2025-01-06 23:42:00-05:00', '2025-01-06 23:39:00-05:00',
               '2025-01-06 23:36:00-05:00', '2025-01-06 23:33:00-05:00',
               ...
               '2025-01-27 01:05:00-05:00', '2025-01-27 00:55:00-05:00',
               '2025-01-27 00:50:00-05:00', '2025-01-27 00:40:00-05:00',
               '2025-01-27 00:35:00-05:00', '2025-01-27 00:25:00-05:00',
               '2025-01-27 00:20:00-05:00', '2025-01-27 00:10:00-05:00',
               '2025-01-27 00:05:00-05:00', '2025-01-28 16:57:01-05:00'],
              dtype='datetime64[ns, America/Bogota]', name='datetime', length=24491, freq=None)